In [6]:
import urllib.request
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import numpy as np
import datetime
from urllib.request import urlopen
import requests
import time

In [7]:
chromedriver_dir = r'C:\Users\LG\빅데이터 실습\chromedriver_win32/chromedriver.exe'
driver = webdriver.Chrome(chromedriver_dir)

In [8]:
# 위키백과로 진행
url = 'https://ko.wikipedia.org/wiki/%EC%95%84%EB%8A%94_%ED%98%95%EB%8B%98#%EC%8B%9C%EC%B2%AD%EB%A5%A0'

driver.get(url)
time.sleep(3)

In [9]:
# 조회수 크롤링에 맞게 가져오기 위한
# 시청률 펼치기 작업 진행

# 2020년도 시청률 펼치기
xpath = """//*[@id="mw-content-text"]/div[1]/div[7]/span/a"""
driver.find_element_by_xpath(xpath).click()

# 2020년도 시청률 펼치기
xpath = """//*[@id="mw-content-text"]/div[1]/div[8]/span/a"""
driver.find_element_by_xpath(xpath).click()

In [10]:
# BeautifulSoup 실행

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
print(soup)

<html class="client-js vector-animations-ready ve-available" dir="ltr" lang="ko"><head>
<meta charset="utf-8"/>
<title>아는 형님 - 위키백과, 우리 모두의 백과사전</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"ko","wgMonthNames":["","1월","2월","3월","4월","5월","6월","7월","8월","9월","10월","11월","12월"],"wgRequestId":"f510a05f-f2e8-40ce-b973-772c239a4804","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"아는_형님","wgTitle":"아는 형님","wgCurRevisionId":30737985,"wgRevisionId":30737985,"wgArticleId":1405438,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["JTBC의 예능 프로그램","대한민국의 텔레비전 버라이어티 쇼","대한민국의 텔레비전 퀴즈 쇼","2015년에 시작한 대한민국 TV 프로그램","2010년대 대한민국의 텔레비전 프로그램","2020년대 대한민국의 텔레비전 프로그램"],"wgPageContentLanguage":"ko",
"wgPageContentModel":"wikitext","wgRelevantPageName":"아는

In [34]:
viewr_ratings = [] # 시청률 넣을 리스트
episode_list = [] # 회차 넣을 리스트

In [35]:
# 2021년도 시청률 가져옴
xpath_basic = """//*[@id="mw-content-text"]/div[1]/div[8]/div[2]/table/tbody/tr[""" # xpath 기본

for i in np.arange(2, 47):
    # 회차
    episode_tmp = str(i) + "]/td[1]" # 각 회차에 얻어오가 위함
    episode_xpath = xpath_basic + episode_tmp # 회차 얻어오기 위한 xpath값 구함
    
    find_element = driver.find_element_by_xpath(episode_xpath)
    episode = find_element.get_attribute('innerHTML')
    episode = episode.replace("회\n", "")
    episode_list.append(episode)
    
    # 시청률
    viewr_tmp = str(i) + "]/td[3]" # 각 시청률에 얻어오가 위함
    viewr_xpath = xpath_basic + viewr_tmp # 시청률 얻어오기 위한 xpath값 구함
    
    find_element = driver.find_element_by_xpath(viewr_xpath)
    viewr = find_element.get_attribute('innerHTML')
    viewr = viewr.replace("\n", "")
    viewr_ratings.append(viewr)

In [36]:
# 2020년도 시청률 가져옴
xpath_basic = """//*[@id="mw-content-text"]/div[1]/div[7]/div[2]/table/tbody/tr[""" # xpath 기본

for i in np.arange(43, 51):
    # 회차
    episode_tmp = str(i) + "]/td[1]" # 각 회차에 얻어오가 위함
    episode_xpath = xpath_basic + episode_tmp # 회차 얻어오기 위한 xpath값 구함
    
    find_element = driver.find_element_by_xpath(episode_xpath)
    episode = find_element.get_attribute('innerHTML')
    episode = episode.replace("회\n", "")
    episode_list.append(episode)
    
    # 시청률
    viewr_tmp = str(i) + "]/td[3]" # 각 시청률에 얻어오가 위함
    viewr_xpath = xpath_basic + viewr_tmp # 시청률 얻어오기 위한 xpath값 구함
    
    find_element = driver.find_element_by_xpath(viewr_xpath)
    viewr = find_element.get_attribute('innerHTML')
    viewr = viewr.replace("\n", "")
    viewr_ratings.append(viewr)

In [43]:
# 회차, 시청률 저장할 데이터프레임
knowingbrothers_viewr = pd.DataFrame(columns=['회차', '시청률', '회차_merge용'])
knowingbrothers_viewr

,회차,시청률,회차_merge용


In [44]:
# 데이터프레임에 저장
for i in np.arange(0, len(viewr_ratings)):
    merge_episode = "아는 형님 " + str(episode_list[i]) + "회"
    dic = {"회차" : episode_list[i],
           "시청률" : viewr_ratings[i],
           "회차_merge용" : merge_episode}
    
    knowingbrothers_viewr = knowingbrothers_viewr.append(dic, ignore_index=True)

# 시청률 순으로 내림차순 정렬
knowingbrothers_viewr = knowingbrothers_viewr.sort_values('회차', ascending=False).reset_index(drop=True)
knowingbrothers_viewr

,회차,시청률,회차_merge용
0,306,2.3%,아는 형님 306회
1,305,2.3%,아는 형님 305회
2,304,2.4%,아는 형님 304회
3,303,1.9%,아는 형님 303회
4,302,2.5%,아는 형님 302회
5,301,2.6%,아는 형님 301회
6,300,3.0%,아는 형님 300회
7,299,2.6%,아는 형님 299회
8,298,2.0%,아는 형님 298회
9,297,1.9%,아는 형님 297회


In [45]:
# 시청률 csv 파일로 저장
knowingbrothers_viewr.to_csv('knowingbrothers_viewr.csv', index=False, encoding="utf-8-sig")

In [46]:
# 네이버 tv 크롤링 결과 가져오기
knowingbrothers_view = pd.read_csv('knowingbrothers_view.csv')
knowingbrothers_view.rename(columns = {"회차" : "회"}, inplace=True)
knowingbrothers_view

,회,영상 이름,조회수,좋아요 수
0,아는 형님 306회,뮤지컬 여신 옥주현💃🏻&A+급 성질머리🔥 장동..,6988,47
1,아는 형님 306회,"3人3色 매력 뿜뿜🌈 형님들의 절친, 주우재&다..",5432,79
2,아는 형님 306회,빛나는 친구들 두둥등장🌟 장준&설현&시우민 We..,9050,254
3,아는 형님 306회,세상 섹시한 타로점🔮(¬◡¬)✧ 김희철x장준의 ＜U＞..,5985,161
4,아는 형님 306회,"김영철 또 짝사랑 중,, 비 오는 날☔️ 옥주현에..",4002,35
...,...,...,...,...
1271,아는 형님 253회,처음 문제 맞혀 신난 (여자)아이들 우기 x 미연..,31147,530
1272,아는 형님 253회,(다 같이 뿜 UP👍🏻) 모모랜드 주이 x 낸시의 ..,19389,124
1273,아는 형님 253회,(울컥😥) 위키미키 완전체로 나올 생각에 눈물 ..,49679,820
1274,아는 형님 253회,[선공개] (🐶소리 대결) 채경vs지수! 의도치 않..,22792,357


In [47]:
knowingbrothers = pd.merge(knowingbrothers_viewr, knowingbrothers_view, left_on='회차_merge용', right_on='회')
knowingbrothers = knowingbrothers.drop(['회차_merge용', '회'], axis=1)
knowingbrothers

,회차,시청률,영상 이름,조회수,좋아요 수
0,306,2.3%,뮤지컬 여신 옥주현💃🏻&A+급 성질머리🔥 장동..,6988,47
1,306,2.3%,"3人3色 매력 뿜뿜🌈 형님들의 절친, 주우재&다..",5432,79
2,306,2.3%,빛나는 친구들 두둥등장🌟 장준&설현&시우민 We..,9050,254
3,306,2.3%,세상 섹시한 타로점🔮(¬◡¬)✧ 김희철x장준의 ＜U＞..,5985,161
4,306,2.3%,"김영철 또 짝사랑 중,, 비 오는 날☔️ 옥주현에..",4002,35
...,...,...,...,...,...
1250,253,3.7%,처음 문제 맞혀 신난 (여자)아이들 우기 x 미연..,31147,530
1251,253,3.7%,(다 같이 뿜 UP👍🏻) 모모랜드 주이 x 낸시의 ..,19389,124
1252,253,3.7%,(울컥😥) 위키미키 완전체로 나올 생각에 눈물 ..,49679,820
1253,253,3.7%,[선공개] (🐶소리 대결) 채경vs지수! 의도치 않..,22792,357
